In [1]:
def calculate_md5(*args, __read_path_object__: bool = False, **kwargs) -> str:
    """
    메모리에 담긴 데이터들을 모두 모아 MD5 hash를 만드는 함수

    :param args: positional arguments to be hashed
    :param __read_path_object__: Path 인스턴스의 처리 방법을 결정
    False (기본값): Path 인스턴스, 즉, Python 프로세스의 메모리 값을 hash에 사용
    True: Path 인스턴스가 가리키는 파일에 담긴 디스크 값을 hash에 사용. 파일은 있을 것으로 가정. 파일이 없으면 FileNotFoundError 예외 발생.
    :param kwargs: keyword arguments
    :return: MD5 hash 문자열
    """
    import hashlib
    import pickle
    from pathlib import Path
    import pandas as pd

    def __polish_arg__(arg) -> bytes:
        if isinstance(arg, (pd.DataFrame, pd.Series)):  # pandas Dataframe needs attention.
            arg = pd.util.hash_pandas_object(arg)  # pd.Dataframe or pd.Series becomes pd.Series.
            arg = (arg.index.values, arg.values)  # make a series a tuple containing indices and values

        if __read_path_object__ and isinstance(arg, Path):
            arg = arg.read_bytes()  # (if necessary) read file content as bytes
        if isinstance(arg, str):
            arg = arg.encode()  # str to bytes
        elif not isinstance(arg, bytes):
            arg = pickle.dumps(arg)  # any other object to bytes
        return arg  # it must be bytes.

    m = hashlib.md5()
    for arg in args:
        arg = __polish_arg__(arg)
        m.update(arg)
    for key, value in kwargs.items():
        m.update(__polish_arg__(key))
        m.update(__polish_arg__(value))
    return m.hexdigest()


print(calculate_md5())  # md5hash의 초기 해쉬 d41d8cd98f00b204e9800998ecf8427e
print(calculate_md5(1))  # 객체 1에 대한 해쉬 35a5b1d85b7654229dfe85526c3f385f
print(calculate_md5(1))  # 객체 1에 대한 해쉬 35a5b1d85b7654229dfe85526c3f385f    -> 같은 입력에 대해 언제나 같은 출력 보장
print(calculate_md5(1, 2))  # 객체 1과 2에 대한 해쉬 9446eac0a003b726e6aa15660c543171
print(calculate_md5([1, 2]))  # 길이 2의 리스트 객체 하나에 대한 해쉬 39bb505082b15a2c2c30226da93ef0ee

print(calculate_md5(one=1))  # 키워드 인자 {'one': 1}에 대한 해쉬 d7d8a0e0d2cb6b71bf950734c9c505d6
print(calculate_md5(two=1))  # 키워드 인자 {'two': 1}에 대한 해쉬 1d4896843332d5b3fe38bb80d4f9b999

from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
print(calculate_md5(df))  # iris dataset이 담긴 dataframe에 대한 해쉬 3e60f9a78219f47d36fdc3d84753be5b

d41d8cd98f00b204e9800998ecf8427e
35a5b1d85b7654229dfe85526c3f385f
35a5b1d85b7654229dfe85526c3f385f
9446eac0a003b726e6aa15660c543171
39bb505082b15a2c2c30226da93ef0ee
d7d8a0e0d2cb6b71bf950734c9c505d6
1d4896843332d5b3fe38bb80d4f9b999
3e60f9a78219f47d36fdc3d84753be5b


In [2]:
from pathlib import Path
import pickle # 파이썬 메모리를 덤프할 수 있도록 해주는 유틸리티
from tqdm import trange

path_cache = Path('./cache')
path_cache.mkdir(exist_ok=True)

def load_large_dataset(argument):
    """정말 오래 걸리는 작업"""
    
    md5hash = calculate_md5(argument) # 함수의 실행 결과에 영향을 미칠말한 요소를 모두 해싱에 활용할 것
    
    # 캐쉬 재활용 
    path = path_cache.joinpath(f'{md5hash}.pkl')
    if path.exists():
        return pickle.loads(path.read_bytes())

    # 캐쉬가 없다. 재활용
    iris = load_iris()
    df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
    df['target'] = iris.target
    df['TEST_COLUMN_FOR_CACHING_EXPERIMENT'] = argument
    for _ in trange(200_000_000): continue # 무언가 오래 걸리는 작업이 있다고 치자.

    # 캐쉬를 만든다.
    path.write_bytes(pickle.dumps(df))
    
    # 함수의 본래 출력을 반환한다. 
    return df
    
df = load_large_dataset(0)

100%|███████████████████████| 200000000/200000000 [00:07<00:00, 27173261.17it/s]


In [3]:
df = load_large_dataset(0) # 두 번째 실행은 빠르다.
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,TEST_COLUMN_FOR_CACHING_EXPERIMENT
0,5.1,3.5,1.4,0.2,0,0
1,4.9,3.0,1.4,0.2,0,0
2,4.7,3.2,1.3,0.2,0,0
3,4.6,3.1,1.5,0.2,0,0
4,5.0,3.6,1.4,0.2,0,0
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,0
146,6.3,2.5,5.0,1.9,2,0
147,6.5,3.0,5.2,2.0,2,0
148,6.2,3.4,5.4,2.3,2,0


In [4]:
df = load_large_dataset(1) # 인자가 바뀌었다. 첫 번째 실행이다.
df

100%|███████████████████████| 200000000/200000000 [00:07<00:00, 28349854.57it/s]


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,TEST_COLUMN_FOR_CACHING_EXPERIMENT
0,5.1,3.5,1.4,0.2,0,1
1,4.9,3.0,1.4,0.2,0,1
2,4.7,3.2,1.3,0.2,0,1
3,4.6,3.1,1.5,0.2,0,1
4,5.0,3.6,1.4,0.2,0,1
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,1
146,6.3,2.5,5.0,1.9,2,1
147,6.5,3.0,5.2,2.0,2,1
148,6.2,3.4,5.4,2.3,2,1


In [5]:
df = load_large_dataset(1) # 두 번째 실행이다.
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,TEST_COLUMN_FOR_CACHING_EXPERIMENT
0,5.1,3.5,1.4,0.2,0,1
1,4.9,3.0,1.4,0.2,0,1
2,4.7,3.2,1.3,0.2,0,1
3,4.6,3.1,1.5,0.2,0,1
4,5.0,3.6,1.4,0.2,0,1
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,1
146,6.3,2.5,5.0,1.9,2,1
147,6.5,3.0,5.2,2.0,2,1
148,6.2,3.4,5.4,2.3,2,1


In [6]:
# 캐쉬 디렉터리에 담긴 캐쉬 파일들
!ls -al cache/

total 24
drwxrwxr-x 2 sallysooo sallysooo 4096 Jun 23 11:09 .
drwxrwxr-x 4 sallysooo sallysooo 4096 Jun 23 11:09 ..
-rw-rw-r-- 1 sallysooo sallysooo 8122 Jun 23 11:09 35a5b1d85b7654229dfe85526c3f385f.pkl
-rw-rw-r-- 1 sallysooo sallysooo 8122 Jun 23 11:09 f6b903077fadd6f7f7924a2dcfd0d828.pkl
